In [1]:
import os
import argparse
import time

import pystac_client
import planetary_computer
import stackstac

import geopandas as gpd
import numpy as np

import rasterio
from rasterio.merge import merge
from rasterio import mask
from rasterio import plot
import shapely
import ast
import pandas as pd

In [ ]:
More targeted assessments at known industrial sites
where height estimation is less than 1, we ignore the height,

In [7]:
predictions_dir="/hpc/group/borsuklab/csr33/object_detection/predictions"

44

In [2]:
height_estimation_dir="/hpc/group/borsuklab/csr33/object_detection/height_estimation"
height_paths = [os.path.join(height_estimation_dir, f"merged_predictions_height_{i}.parquet") for i in range(173)]
# Open each Parquet file as a PyArrow table
tables = [gpd.read_parquet(f) for f in height_paths]
detected_tanks_gdf = gpd.GeoDataFrame(pd.concat(tables, ignore_index=True), crs=tables[0].crs)
# Concatenate the PyArrow tables into one table
detected_tanks_gdf = detected_tanks_gdf.sort_values("confidence", ascending=False)

has_height_gdf = detected_tanks_gdf[~detected_tanks_gdf['height'].isnull()]
has_height_gdf = has_height_gdf[has_height_gdf["height"] > 0.0]#.sort_values("height")
has_height_gdf["capacity"] = np.pi * (has_height_gdf["diameter"]/2)**2 * has_height_gdf["height"]
has_height_gdf.loc[has_height_gdf['class_name'] == "spherical_tank", 'capacity'] = None 


In [3]:
complete_predicted_data_dir="/hpc/group/borsuklab/csr33/object_detection/complete_predicted_data"
data_paths = [os.path.join(complete_predicted_data_dir,
                             f"complete_predictions_{i}.parquet") for i in range(10)]
# Open each Parquet file as a PyArrow table
tables = [gpd.read_parquet(f) for f in data_paths]
detected_tanks_gdf = gpd.GeoDataFrame(pd.concat(tables, ignore_index=True), crs=tables[0].crs)
# Concatenate the PyArrow tables into one table
detected_tanks_gdf = detected_tanks_gdf.sort_values("confidence", ascending=False)

In [7]:
rewrite to do capacity for complete predicted data?
where did specific gravity go?

,confidence,class_name,bbox_pixel_coords,tile_names,utm_coords,diameter,utm_proj,geometry,height,chemical_name,cas_number,facility_name
319,"[0.9214309453964233, 0.6490179896354675]",['closed_roof_tank'],"[11499, 14465, 11661, 14621]",m_3009122_ne_15_030_20211116,"[664775.85, 3399064.35, 664824.4500000001, 339...",46.8,EPSG:26915,"POLYGON ((-91.27874 30.71304, -91.27874 30.712...",16.500000,"['Hydrogen sulfide', 'Ethylbenzene', 'Toluene'...","['7783-06-4', '100-41-4', '108-88-3', '1330-20...",[PORT HUDSON TERMINAL]
249,"[0.9202601313591003, 0.778704822063446]",['closed_roof_tank'],"[13418, 16036, 13550, 16152]",m_3009122_ne_15_030_20211116,"[665351.55, 3398593.0500000003, 665391.15, 339...",34.8,EPSG:26915,"POLYGON ((-91.27290 30.70871, -91.27290 30.708...",15.720000,"['Hydrogen sulfide', 'Ethylbenzene', 'Toluene'...","['7783-06-4', '100-41-4', '108-88-3', '1330-20...",[PORT HUDSON TERMINAL]
250,"[0.914099931716919, 0.9086913466453552]",['closed_roof_tank'],"[13439, 16534, 13620, 16743]",m_3009122_ne_15_030_20211116,"[665357.85, 3398443.65, 665412.15, 3398380.95]",54.3,EPSG:26915,"POLYGON ((-91.27271 30.70736, -91.27271 30.706...",14.760000,"['Hydrogen sulfide', 'Ethylbenzene', 'Toluene'...","['7783-06-4', '100-41-4', '108-88-3', '1330-20...",[PORT HUDSON TERMINAL]
251,"[0.9115021228790283, 0.8991819620132446]",['closed_roof_tank'],"[13347, 16234, 13572, 16442]",m_3009122_ne_15_030_20211116,"[665330.25, 3398533.65, 665397.75, 3398471.25]",62.4,EPSG:26915,"POLYGON ((-91.27285 30.70817, -91.27285 30.707...",14.760000,"['Hydrogen sulfide', 'Ethylbenzene', 'Toluene'...","['7783-06-4', '100-41-4', '108-88-3', '1330-20...",[PORT HUDSON TERMINAL]
273,"[0.910485029220581, 0.8887038230895996]",['closed_roof_tank'],"[12302, 13366, 12429, 13482]",m_3009122_ne_15_030_20211116,"[665016.75, 3399394.0500000003, 665054.85, 339...",34.8,EPSG:26915,"POLYGON ((-91.27628 30.71598, -91.27628 30.715...",15.690000,"['Hydrogen sulfide', 'Ethylbenzene', 'Toluene'...","['7783-06-4', '100-41-4', '108-88-3', '1330-20...",[PORT HUDSON TERMINAL]
...,...,...,...,...,...,...,...,...,...,...,...,...
637,0.5014625191688538,closed_roof_tank,"[8686, 22518, 8693, 22525]",m_3109062_sw_15_030_20211120,"[752780.8500000001, 3433233.75, 752782.9500000...",2.1,EPSG:26915,"POLYGON ((-90.35241 31.00561, -90.35241 31.005...",NaN,None,None,None
993,0.5012210607528687,closed_roof_tank,"[8186, 485, 8194, 493]",m_3308062_nw_17_060_20210426,"[563013.9, 3665264.7, 563018.7000000001, 36652...",4.8,EPSG:26917,"POLYGON ((-80.32445 33.12425, -80.32445 33.124...",NaN,None,None,None
854,0.501176118850708,closed_roof_tank,"[271, 24061, 283, 24071]",m_3009034_ne_15_030_20211120,"[709821.4500000001, 3369329.5500000003, 709825...",3.0,EPSG:26915,"POLYGON ((-90.81507 30.43783, -90.81507 30.437...",18.610001,None,None,None
390,0.5011230707168579,closed_roof_tank,"[4551, 5386, 4559, 5394]",m_2608040_sw_17_060_20191120_20200113,"[589770.9, 2921444.1, 589775.7000000001, 29214...",4.8,EPSG:26917,"POLYGON ((-80.09981 26.41036, -80.09981 26.410...",NaN,['Zinc compounds'],['N982'],[NHS U.S. LLC]


In [83]:
has_height_gdf

,confidence,class_name,bbox_pixel_coords,tile_names,utm_coords,diameter,utm_proj,geometry,height,capacity
15715,"[0.939806342124939, 0.7305512428283691]",['closed_roof_tank'],"[4350, 3556, 4469, 3650]",m_2909006_sw_15_030_20211130,"[754485.15, 3314029.0500000003, 754520.85, 331...",28.2,EPSG:26915,"POLYGON ((-90.36336 29.93070, -90.36336 29.930...",8.280000,5171.522527
20986,"[0.9394638538360596, 0.8525862693786621]",['closed_roof_tank'],"[7655, 9606, 7773, 9721]",m_2909515_se_15_060_20201130,"[292857.3, 3294452.1, 292928.1, 3294383.1]",69.0,EPSG:26915,"POLYGON ((-95.14165 29.76304, -95.14165 29.762...",27.370001,102344.114691
3400,"[0.9393055438995361, 0.8024093508720398]",['closed_roof_tank'],"[4834, 1903, 4914, 1976]",m_2909309_nw_15_060_20201102,"[406070.7, 3304413.9000000004, 406118.7, 33043...",43.8,EPSG:26915,"POLYGON ((-93.97207 29.86671, -93.97207 29.866...",15.969999,24062.624830
23226,"[0.9361604452133179, 0.5756282806396484]",['closed_roof_tank'],"[1360, 8251, 1443, 8329]",m_2909523_nw_15_060_20201130,"[282900.3, 3288455.1, 282950.1, 3288408.300000...",46.8,EPSG:26915,"POLYGON ((-95.24359 29.70725, -95.24359 29.706...",15.836000,27241.253162
4848,"[0.9354996681213379, 0.9061532616615295]",['closed_roof_tank'],"[6727, 4444, 6828, 4542]",m_2909512_sw_15_060_20201030,"[249982.5, 3298431.3000000003, 250043.09999999...",58.8,EPSG:26915,"POLYGON ((-95.58577 29.79099, -95.58577 29.790...",7.620000,20691.858428
...,...,...,...,...,...,...,...,...,...,...
6709,0.5002272725105286,closed_roof_tank,"[2430, 1961, 2445, 1976]",m_4107205_nw_18_060_20211001,"[708209.7000000001, 4652076.9, 708218.70000000...",9.0,EPSG:26918,"POLYGON ((-72.48617 41.99319, -72.48617 41.993...",3.332000,211.972679
8929,0.500196635723114,closed_roof_tank,"[1538, 7964, 1547, 7972]",m_3807531_nw_18_060_20210624,"[478943.1, 4270617.3, 478948.5, 4270612.5]",4.8,EPSG:26918,"POLYGON ((-75.24170 38.58378, -75.24170 38.583...",6.800000,123.049904
29911,0.5001552104949951,external_floating_roof_tank,"[4178, 3411, 4195, 3427]",m_4406928_sw_19_060_20210904,"[452521.5, 4932849.9, 452531.7, 4932840.3]",9.6,EPSG:26919,"POLYGON ((-69.59758 44.54742, -69.59758 44.547...",1.464000,105.967678
11245,0.5001063346862793,closed_roof_tank,"[9850, 7522, 9859, 7530]",m_2909442_sw_15_060_20201116,"[323508.3, 3239752.5, 323513.7, 3239747.7]",4.8,EPSG:26915,"POLYGON ((-94.81664 29.27437, -94.81664 29.274...",6.798800,123.028189


In [76]:
class_name_merge = ['class1', 'class2'] 
conf = [0.7, 0.9]

max_conf_idx = np.argmax(conf)
max_conf_class = class_name_merge[max_conf_idx]

print(max_conf_class)

class2


In [77]:
np.mean(conf)

0.8

In [75]:
has_height_gdf.class_name.unique()

array(["['closed_roof_tank']", "['external_floating_roof_tank']",
       "['closed_roof_tank' 'external_floating_roof_tank']",
       "['spherical_tank']", "['closed_roof_tank' 'spherical_tank']",
       'closed_roof_tank', 'external_floating_roof_tank',
       'spherical_tank'], dtype=object)

In [30]:
detected_tanks = gpd.read_parquet(os.path.join("/hpc/group/borsuklab/csr33/object_detection/height_estimation", f"merged_predictions_height_{0}.parquet"))

In [20]:
print(args)
detected_tanks = gpd.read_parquet(os.path.join(args.prediction_dir, f"merged_predictions_{args.chunk_id}.parquet"))
#reformat     
detected_tanks['utm_coords'] = detected_tanks['utm_coords'].apply(lambda x: ast.literal_eval(x))

Namespace(prediction_dir='/hpc/group/borsuklab/csr33/object_detection/predictions', collection='3dep-lidar-hag', chunk_id=0, backoff_factor=10, max_retries=10, imgsz=640, model_path='/work/csr33/object_detection/runs/detect/train_w_tuned_hyperparameters3/weights/best.pt', tile_dir='/hpc/group/borsuklab/csr33/object_detection/naip_tiles')


In [13]:
detected_tanks["height"] = height_estimation(detected_tanks, args)

Namespace(prediction_dir='/hpc/group/borsuklab/csr33/object_detection/predictions', collection='3dep-lidar-hag', chunk_id=0, backoff_factor=10, max_retries=10, imgsz=640, model_path='/work/csr33/object_detection/runs/detect/train_w_tuned_hyperparameters3/weights/best.pt', tile_dir='/hpc/group/borsuklab/csr33/object_detection/naip_tiles')
<class 'NameError'> handle
retry
Failed, retrying in 1 seconds...
<class 'NameError'> handle
retry
Failed, retrying in 2 seconds...
<class 'NameError'> handle
Namespace(prediction_dir='/hpc/group/borsuklab/csr33/object_detection/predictions', collection='3dep-lidar-hag', chunk_id=0, backoff_factor=10, max_retries=10, imgsz=640, model_path='/work/csr33/object_detection/runs/detect/train_w_tuned_hyperparameters3/weights/best.pt', tile_dir='/hpc/group/borsuklab/csr33/object_detection/naip_tiles')
<class 'NameError'> handle
retry
Failed, retrying in 1 seconds...


KeyboardInterrupt: 

In [21]:
for i, row in detected_tanks.iterrows():
    break

In [24]:
    catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1",
                                        modifier = planetary_computer.sign_inplace,)

In [26]:
height_estimation_by_tank(row, catalog, args)

In [ ]:
# Get the arguments
detected_tanks['utm_coords'] = detected_tanks['utm_coords'].apply(lambda x: str(x))

detected_tanks.to_parquet(os.path.join(args.prediction_dir, f"merged_predictions_height_{args.chunk_id}.parquet"))